In [1]:
import pandas as pd

In [2]:
file = pd.read_csv('track_with_genre.csv')

FileNotFoundError: File b'track_with_genre.csv' does not exist

In [8]:
genre_MUSIC_values = [
    
    {
        'genre': 'Soft rock',
        'M':  .33,
        'U':  .07,
        'S':  -.06, 
        'I':  -.1,
        'C':  -.12
    },
    {
        'genre': 'RnB',
        'M':  .31,
        'U':  .04,
        'S':  -.11, 
        'I':  -.1,
        'C':  .06
    }, 
    {
        'genre': 'Country',
        'M':  0,
        'U':  .36,
        'S':  -.2, 
        'I':  -.12,
        'C':  -.09
    },     
    {
        'genre': 'Bluegrass',
        'M':  -.11,
        'U':  .33,
        'S':  0, 
        'I':  -.11,
        'C':  -.16
    },  
    {
        'genre': 'Pop_Rock',
        'M':  -.09,
        'U':  .17,
        'S':  -.06, 
        'I':  -.04,
        'C':  .02
    }, 
    {
        'genre': 'Classical',
        'M':  .04,
        'U':  -.19,
        'S':  .37, 
        'I':  -.09,
        'C':  -.19
    }, 
    {
        'genre': 'Jazz',
        'M':  .04,
        'U':  -.13,
        'S':  .15, 
        'I':  -.12,
        'C':  .1
    },    
    {
        'genre': 'Pop',
        'M':  .02,
        'U':  -.07,
        'S':  -.1, 
        'I':  .22,
        'C':  -.04
    },    
    {
        'genre': 'Rap',
        'M':  -.17,
        'U':  -.2,
        'S':  -.25, 
        'I':  -.06,
        'C':  .51
    },        
    {
        'genre': 'Latin',
        'M':  .01,
        'U':  -.09,
        'S':  .14, 
        'I':  -.11,
        'C':  .20
    }
    
]

In [ ]:
OCEAN_MUSIC_values = [
    
    # O
    [-.12, -.02,.16, .07, -.11],
    
    # C
    [.05, .02, -.03,  -.02, 0],
    
    # E
    [ -.05, .13, -.06, 0, -.01],

    # A
    [.06, 0, -.06,  0, .02],
    
    # N
    [.03, -.06, .02, .04, 0]
]

In [14]:
# get all songs for a user

def get_user_genre_plays(user_id):
    user_songs = file.loc[file['UserID'] == user_id]
    
    return user_songs[['NumPlays','Genre']]

genre_plays = get_user_genre_plays('f926327f2c99580a4aa414d1cfc0439f45a929bf')

print(genre_plays)

          NumPlays          Genre
2                5       Pop_Rock
35575            2       Pop_Rock
43076           11       Pop_Rock
45267           10       Pop_Rock
258121           6       Pop_Rock
265426          13        Country
370798           1       Pop_Rock
584632           1       Pop_Rock
752425           3     Electronic
2308021          2       Pop_Rock
2430657          4        Country
2698733         13       Pop_Rock
2722858          2       Pop_Rock
2724193          1       Pop_Rock
2797248          5       Pop_Rock
3096531          7       Pop_Rock
3707016          6       Pop_Rock
4571982          6       Pop_Rock
4584786          3       Pop_Rock
4842061          7       Pop_Rock
4868050          1       Pop_Rock
4979839          3       Pop_Rock
5074291          2       Pop_Rock
5201337          2       Pop_Rock
5350282         38       Pop_Rock
5476646          2       Pop_Rock
6296892          3       Pop_Rock
6319737          5       Pop_Rock
6351348       

In [16]:
# get percentage genre counts for the user (by number of plays per genre, not number of unique songs)

def get_genre_percentages(user_songs):
    
    genre_dict = {}
    total_plays = 0
    
    for index, row in user_songs.iterrows():
        if row['Genre'] not in genre_dict:
            genre_dict[row['Genre']] = row['NumPlays']
            total_plays += row['NumPlays']
        else:
            genre_dict[row['Genre']] += row['NumPlays']
            total_plays += row['NumPlays']
            
    for key in genre_dict:
        genre_dict[key] = 1.0*genre_dict[key]/total_plays
            
    return genre_dict

genre_percentages = get_genre_percentages(genre_plays)

print(genre_percentages)

{'Pop_Rock': 0.7870182555780934, 'Country': 0.0973630831643002, 'Electronic': 0.034482758620689655, 'RnB': 0.05273833671399594, 'International': 0.018255578093306288, 'Stage ': 0.010141987829614604}


In [18]:
# multiply the percentages by the corresponding MUSIC scores, add them up, thus getting their MUSIC score

valid_genres = ['RnB', 'Country', 'Pop_Rock', 'Classical', 'Jazz', 'Rap', 'Latin']    

def get_MUSIC_values(genre_percentages):
    
    MUSIC_values = [0,0,0,0,0]
    
    for genre in genre_percentages:
        if genre in valid_genres:
            genre_percentage = genre_percentages[genre]
            
            for MUSIC_dict in genre_MUSIC_values:
                if MUSIC_dict['genre'] == genre:
                    MUSIC_values[0] += MUSIC_dict['M']*genre_percentage
                    MUSIC_values[1] += MUSIC_dict['U']*genre_percentage
                    MUSIC_values[2] += MUSIC_dict['S']*genre_percentage
                    MUSIC_values[3] += MUSIC_dict['I']*genre_percentage
                    MUSIC_values[4] += MUSIC_dict['C']*genre_percentage
                    
    return MUSIC_values

MUSIC_values = get_MUSIC_values(genre_percentages)

print(MUSIC_values)

[-0.05448275862068966, 0.1709533468559838, -0.0724949290060852, -0.04843813387423935, 0.010141987829614604]


In [19]:
# convert MUSIC score into OCEAN score

def convert_MUSIC_to_OCEAN(MUSIC_values):
    
    OCEAN_values = [0, 0, 0, 0, 0]
    
    for i in range(len(OCEAN_values)):
        for j in range(len(MUSIC_values)):
            OCEAN_values[i] += MUSIC_values[j] * OCEAN_MUSIC_values[i][j]

    return OCEAN_values

OCEAN_values = convert_MUSIC_to_OCEAN(MUSIC_values)

print(OCEAN_values)

[-0.012986612576064909, 0.0038385395537525353, 0.02919634888438134, 0.0012835699797160245, -0.015279107505070996]
